In [19]:
import pandas as pd
import gzip
from glob import glob
import geopandas as gpd
import json
import time
from tqdm import tqdm

In [20]:
def checkLeadingZero(val):
    if len(val) == 13:
        return val[0:4]
    else:
        return val[0:5]

In [31]:
months = glob(f'E:/SG_WEEKLY/patterns_backfill/2020/12/14/21/2020/*')
dumpDf = ''

for month in tqdm(months):
    weeks = glob(f'{month}/*')
    for week in tqdm(weeks):
        files = glob(f'{week}/*.gz')
        
        sg_raw = pd.read_csv(files[0])
        for i in range(1, len(files)):
            sg_raw = pd.concat([sg_raw, pd.read_csv(files[i])], ignore_index=True)
        
        date = sg_raw.iloc[0].date_range_end[:10]
        sg_raw.poi_cbg = sg_raw.poi_cbg.astype(str)
        sg_raw['county'] = sg_raw['poi_cbg'].apply(lambda x: checkLeadingZero(x))
        sg_parsed = sg_raw.dropna()[['raw_visit_counts','county']].groupby('county').sum().reset_index()
        sg_parsed.columns=['county', date]
        
        if len(dumpDf) == 0:
            dumpDf = sg_parsed
        else:
            dumpDf = dumpDf.merge(sg_parsed, on="county", how="left")

dumpDf.to_csv('./county_POI_visits.csv', index=False)

 25%|█████████████████████                                                               | 1/4 [01:08<03:24, 68.23s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [57:22<00:00, 312.95s/it]
